凯利准则根据胜率和盈亏比计算最优仓位比例，公式为：f = (p * (b + 1) - 1) / b，其中 f 是仓位比例，p 是胜率，b 是盈亏比。

In [5]:
def kelly_criterion(account_balance, win_probability, reward_risk_ratio):
    """
    计算凯利准则仓位
    :param account_balance: 账户总资金
    :param win_probability: 胜率（0到1之间）
    :param reward_risk_ratio: 盈亏比（正数）
    :return: 仓位比例（%），仓位金额
    """
    if not (0 <= win_probability <= 1) or reward_risk_ratio <= 0:
        return 0, 0  # 无效输入返回0
    f = (win_probability * (reward_risk_ratio + 1) - 1) / reward_risk_ratio
    f = max(0, min(f, 1))  # 限制在0到1之间
    position_size = account_balance * f
    return f * 100, position_size

# 示例
account = 100000  # 账户资金
win_prob = 0.471    # 胜率
rr_ratio = 2.295   # 盈亏比

percent, amount = kelly_criterion(account, win_prob, rr_ratio)
print(f"凯利准则: 仓位比例={percent:.2f}%, 仓位金额=${amount:.2f}")

凯利准则: 仓位比例=24.05%, 仓位金额=$24049.89


基于波动率的仓位管理根据 ATR% 和止损倍数调整仓位，公式为：仓位大小 = (账户资金 × 风险比例) / (价格 × ATR% × 止损倍数)。

In [9]:
def volatility_based_position(account_balance, risk_percent, price, atr_percent, stop_multiplier):
    """
    计算基于波动率的仓位（使用ATR%）
    :param account_balance: 账户总资金
    :param risk_percent: 风险比例（%）
    :param price: 当前价格
    :param atr_percent: ATR%（%）
    :param stop_multiplier: 止损倍数（通常1-2）
    :return: 仓位比例（%），仓位金额
    """
    if risk_percent <= 0 or price <= 0 or atr_percent <= 0 or stop_multiplier <= 0:
        return 0, 0  # 无效输入
    risk_amount = account_balance * (risk_percent / 100)  # 风险金额
    stop_loss = price * (atr_percent / 100) * stop_multiplier  # 止损金额
    shares = risk_amount / stop_loss  # 股数
    position_size = shares * price  # 仓位金额
    position_percent = (position_size / account_balance) * 100
    return position_percent, position_size

# 示例
account = 100000  # 账户资金
risk_pct = 2.0    # 承受风险%
price = 49       # 股票价格
atr_pct = 10.26  # ATR%
stop_mult = 1     # 止损倍ATR%

percent, amount = volatility_based_position(account, risk_pct, price, atr_pct, stop_mult)
print(f"基于波动率: 仓位比例={percent:.2f}%, 仓位金额=${amount:.2f}")

基于波动率: 仓位比例=19.49%, 仓位金额=$19493.18


固定比例仓位管理根据固定风险比例和止损百分比计算仓位，公式为：仓位大小 = (账户资金 × 风险比例) / (价格 × 止损百分比)。

In [10]:
def fixed_fractional_position(account_balance, risk_percent, price, stop_loss_percent):
    """
    计算固定比例仓位
    :param account_balance: 账户总资金
    :param risk_percent: 风险比例（%）
    :param price: 当前价格
    :param stop_loss_percent: 止损百分比（%）
    :return: 仓位比例（%），仓位金额
    """
    if risk_percent <= 0 or price <= 0 or stop_loss_percent <= 0:
        return 0, 0  # 无效输入
    risk_amount = account_balance * (risk_percent / 100)
    stop_loss = price * (stop_loss_percent / 100)
    shares = risk_amount / stop_loss
    position_size = shares * price
    position_percent = (position_size / account_balance) * 100
    return position_percent, position_size

# 示例
account = 100000
risk_pct = 1.0 # 风险比例（%）
price = 49 # 当前价格
stop_loss_pct = 5

percent, amount = fixed_fractional_position(account, risk_pct, price, stop_loss_pct)
print(f"固定比例: 仓位比例={percent:.2f}%, 仓位金额=${amount:.2f}")

固定比例: 仓位比例=20.00%, 仓位金额=$20000.00


结合固定比例仓位管理和凯利准则，通过输入胜率、风险比例和止损百分比，计算目标止盈百分比，用于设置合理的止盈价格。

In [6]:
def calculate_target_profit_percentage(win_rate, risk_ratio, stop_loss_percentage):
    """
    计算目标止盈百分比（风险比例/止损百分比<胜率）
    参数：
        win_rate (float): 胜率 (0 到 1 之间，例如 0.6 表示 60%)
        risk_ratio (float): 风险比例 (例如 0.01 表示 1%)
        stop_loss_percentage (float): 止损百分比 (例如 0.02 表示 2%)
    返回：
        target_profit_percentage (float): 目标止盈百分比 (例如 0.03 表示 3%)
    """
    try:
        # 计算 f = 风险比例 / 止损百分比
        f = risk_ratio / stop_loss_percentage

        print(f"仓位百分比: {f*100:.1f}%")
        
        # 计算盈亏比 b = (p - 1) / (f - p)
        b = (win_rate - 1) / (f - win_rate)
        
        # 计算目标止盈百分比 = b * 止损百分比
        target_profit_percentage = b * stop_loss_percentage
        
        # 确保结果合理（b 应为正数，止盈百分比应为正数）
        if b <= 0 or target_profit_percentage <= 0:
            raise ValueError("计算结果不合理，盈亏比或止盈百分比为负，请检查输入参数")
        
        return target_profit_percentage
    except ZeroDivisionError:
        return "错误：除数为零，请检查输入参数（风险比例 / 止损百分比 != 胜率）"
    except ValueError as e:
        return f"错误：{str(e)}"

# 示例用法
win_rate = 0.63
risk_ratio = 0.01
stop_loss_percentage = 0.06

result = calculate_target_profit_percentage(win_rate, risk_ratio, stop_loss_percentage)
if isinstance(result, str):
    print(result)
else:
    print(f"凯利公式目标止盈百分比： {result*100:.2f}%")

仓位百分比: 16.7%
目标止盈百分比： 4.79%
